<a href="https://colab.research.google.com/github/Manas110901100/Calculus/blob/main/ANN's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim


In [8]:
class MyFirstNetwork(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(MyFirstNetwork, self).__init__()
    self.layer_1 = nn.Linear(in_features=input_dim, out_features=output_dim )
    self.Relu = nn.ReLU()
    self.layer_2 = nn.Linear(in_features=output_dim, out_features= 16)
    self.layer_3 = nn.Linear(in_features=16, out_features=1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.layer_1(x)
    out = self.Relu(out)
    out = self.layer_2(out)
    out = self.Relu(out)
    out = self.layer_3(out)
    out = self.sigmoid(out)
    return out



In [18]:
class RegularizedNetwork(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
    self.normalize = nn.BatchNorm1d(hidden_dim)
    self.Relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.5)
    self.layer_2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
    self.layer_3 = nn.Linear(in_features=hidden_dim, out_features=1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    out = self.layer_1(x)
    out = self.normalize(out)
    out = self.Relu(out)
    out = self.dropout(out)
    out = self.layer_2(out)
    out = self.normalize(out)
    out = self.Relu(out)
    out = self.dropout(out)
    out = self.layer_3(out)
    out = self.sigmoid(out)
    return out

In [10]:
X,y = make_classification(n_samples=1000, n_features=100, n_classes=2, random_state=42)

In [11]:

X_test = torch.tensor(X[:700],dtype= torch.float32)
X_train = torch.tensor(X[700:],dtype= torch.float32)
y_test = torch.tensor(y[:700],dtype= torch.float32).unsqueeze(1)
y_train = torch.tensor(y[700:],dtype= torch.float32).unsqueeze(1)

In [29]:
print("--- Training Simple Model ---")
model = MyFirstNetwork(input_dim=100, output_dim=32)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01) # Increased LR slightly

for epoch in range(100):
    predicted = model(X_train)
    loss_value = criterion(predicted, y_train)
    optimizer.zero_grad()
    loss_value.backward()
    optimizer.step()

    if epoch % 10 == 0: # Print every 10 epochs to reduce clutter
        acc = predicted.round().eq(y_train).sum() / y_train.shape[0]
        print(f"Epoch {epoch}: Loss {loss_value.item():.4f}, Acc {acc:.2f}")

# --- MODEL 2: REGULARIZED (Dropout + BatchNorm) ---
print("\n--- Training Regularized Model ---")
reg_model = RegularizedNetwork(input_dim=100, hidden_dim=32)
criterion = nn.BCELoss()

# FIX 1: Pass the CORRECT parameters to the optimizer!
optimizer = optim.Adam(reg_model.parameters(), lr=0.01)

# FIX 2: Set to Train Mode (Enables Dropout)
reg_model.train()

for epoch in range(100):
    predicted = reg_model(X_train)
    loss_value = criterion(predicted, y_train)
    optimizer.zero_grad()
    loss_value.backward()
    optimizer.step()

    if epoch % 10 == 0:
        acc = predicted.round().eq(y_train).sum() / y_train.shape[0]
        print(f"Epoch {epoch}: Loss {loss_value.item():.4f}, Acc {acc:.2f}")

# --- FINAL EVALUATION ---
print("\n--- Final Test Set Results ---")

--- Training Simple Model ---
Epoch 0: Loss 0.6943, Acc 0.52
Epoch 10: Loss 0.3469, Acc 0.92
Epoch 20: Loss 0.0163, Acc 1.00
Epoch 30: Loss 0.0003, Acc 1.00
Epoch 40: Loss 0.0000, Acc 1.00
Epoch 50: Loss 0.0000, Acc 1.00
Epoch 60: Loss 0.0000, Acc 1.00
Epoch 70: Loss 0.0000, Acc 1.00
Epoch 80: Loss 0.0000, Acc 1.00
Epoch 90: Loss 0.0000, Acc 1.00

--- Training Regularized Model ---
Epoch 0: Loss 0.7260, Acc 0.50
Epoch 10: Loss 0.4146, Acc 0.84
Epoch 20: Loss 0.1983, Acc 0.95
Epoch 30: Loss 0.1191, Acc 0.97
Epoch 40: Loss 0.0850, Acc 0.97
Epoch 50: Loss 0.0436, Acc 0.99
Epoch 60: Loss 0.0313, Acc 1.00
Epoch 70: Loss 0.0322, Acc 0.99
Epoch 80: Loss 0.0330, Acc 1.00
Epoch 90: Loss 0.0286, Acc 0.99

--- Final Test Set Results ---


In [24]:
Regularized_model = RegularizedNetwork(input_dim=100, hidden_dim=32)
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(100):
  predicted = Regularized_model(X_train)
  loss_value = loss(predicted, y_train)
  optimizer.zero_grad()
  loss_value.backward()
  optimizer.step()
  predicted_classes = predicted.round()
  accuracy = predicted_classes.eq(y_train).sum() / y_train.shape[0]
  print(f"Epoch: {epoch+1} Loss: {loss_value} Accuracy: {accuracy*100}")

Epoch: 1 Loss: 0.7748419046401978 Accuracy: 43.33333206176758
Epoch: 2 Loss: 0.7257727980613708 Accuracy: 53.66666793823242
Epoch: 3 Loss: 0.753275454044342 Accuracy: 46.66666793823242
Epoch: 4 Loss: 0.7349758148193359 Accuracy: 46.33333206176758
Epoch: 5 Loss: 0.7200084328651428 Accuracy: 48.66666793823242
Epoch: 6 Loss: 0.7452312707901001 Accuracy: 45.333335876464844
Epoch: 7 Loss: 0.7367479205131531 Accuracy: 50.0
Epoch: 8 Loss: 0.7064067125320435 Accuracy: 56.33333206176758
Epoch: 9 Loss: 0.735733687877655 Accuracy: 49.333335876464844
Epoch: 10 Loss: 0.7315855622291565 Accuracy: 48.66666793823242
Epoch: 11 Loss: 0.7301356792449951 Accuracy: 47.0
Epoch: 12 Loss: 0.7182086110115051 Accuracy: 54.000003814697266
Epoch: 13 Loss: 0.7331633567810059 Accuracy: 50.33333206176758
Epoch: 14 Loss: 0.7208091020584106 Accuracy: 52.0
Epoch: 15 Loss: 0.7263095378875732 Accuracy: 49.66666793823242
Epoch: 16 Loss: 0.7311851978302002 Accuracy: 47.666664123535156
Epoch: 17 Loss: 0.7300854325294495 Acc

In [25]:
predicted = Regularized_model(X_test)
predicted_classes = predicted.round()
accuracy = predicted_classes.eq(y_test).sum() / y_test.shape[0]
print(f"Accuracy: {accuracy*100}")

Accuracy: 48.14285659790039


In [26]:
predicted = model(X_test)
predicted_classes = predicted.round()
accuracy = predicted_classes.eq(y_test).sum() / y_test.shape[0]
print(f"Accuracy: {accuracy*100}")

Accuracy: 60.000003814697266


In [30]:
model.eval()
with torch.no_grad():
    pred = model(X_test)
    acc = pred.round().eq(y_test).sum() / y_test.shape[0]
    print(f"Simple Model Test Accuracy:      {acc*100:.2f}%")

reg_model.eval()
with torch.no_grad():
    pred = reg_model(X_test)
    acc = pred.round().eq(y_test).sum() / y_test.shape[0]
    print(f"Regularized Model Test Accuracy: {acc*100:.2f}%")

Simple Model Test Accuracy:      74.43%
Regularized Model Test Accuracy: 78.14%
